import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Bảo hiểm - Trợ cấp xã hội

find elements

In [3]:
# Function to crawl data from a page
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Get author and time information
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Get question title
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Get situation/summary
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Get detail link
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Send HTTP request to get detail page content
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Get answer from detail page
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Add article information to the list
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data

execute function

In [4]:
articles_data = []
for page_num in range(1, 300):
    page_url = f"https://chinhsachonline.chinhphu.vn/bao-hiem-tro-cap-xa-hoi/t1/1/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

export to CSV

In [5]:
# Create DataFrame from the data list
df_bh = pd.DataFrame(articles_data)
df_bh.head(5)

,Name,Time,Question,Situation,Answer
0,Trần Thị Thu Hà,10:05 29/07/2025,Tính đóng BHXH thời gian nghỉ dưỡng sức sau th...,Người lao động nghỉ thai sản sinh con phẫu thu...,Bảo hiểm xã hội khu vực XVIII trả lời vấn đề n...
1,Nguyễn Minh Châu,11:59 28/07/2025,Đối tượng tham gia BHXH tự nguyện,"Bố tôi đã chết, mẹ tôi sinh năm 1969, sống cùn...",Bảo hiểm xã hội khu vực XVIII trả lời vấn đề n...
2,Ngô Doãn Đô,07:05 26/07/2025,Có được hưởng trợ cấp thất nghiệp trước khi xi...,"Tôi sinh tháng 10/1966, đóng BHXH bắt buộc đượ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
3,Vũ Hồng Sang,14:05 21/07/2025,Cách tính lương hưu theo Luật BHXH 2024,"Mẹ tôi là giáo viên THCS, sinh ngày 7/8/1971 v...",Bảo hiểm xã hội khu vực XVI trả lời vấn đề này...
4,Lê Tiến Dũng,14:05 19/07/2025,Ông Lê Tiến Dũng đã được hỗ trợ thủ tục hưởng ...,"Ngày 13/3/2025, tôi đến BHXH TP. Thủ Đức để nộ...",Bảo hiểm xã hội khu vực XXVII trả lời vấn đề n...


In [ ]:
 # Save DataFrame to CSV file
csv_file = '../source_fine_tuning/data/bh_tcxh_data.csv'
df_bh.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../source_fine_tuning/data/bh_tcxh_data.csv


In [7]:
df_bh=pd.read_csv("../source_fine_tuning/data/bh_tcxh_data.csv",encoding='utf-8')

In [8]:
df_bh.shape

(1904, 6)